In [30]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

In [25]:
# Model
m = gp.Model("district")
m

<gurobi.Model Continuous instance district: 0 constrs, 0 vars, Parameter changes: Username=(user-defined)>

In [26]:
state = 'RI'
population = "districting_2020\\2020\\{}\\counties\\graph\\{}.population".format(state, state)


with open('Numberofdistricts.txt', 'r') as file:
    for line in population:
        parts = line.strip().split('\t')
        if parts[0] == state:
            num_districts =  int(parts[1])

with open(population, "r") as file:
    # Skip the first line
    next(file)
    # Count the number of lines, each line represents a county
    num_counties = sum(1 for line in file)



In [29]:
state = 'RI'
adjacency_matrix_file = r"districting_2020\\2020\\{}\\counties\\graph\\{}_distances.csv".format(state, state)

# Set decision variable x for each node j wether it is in district k
for i in range(num_counties):
    for j in range(num_districts):
        x = m.addVar(vtype=gp.GRB.BINARY, name="x_{}_{}".format(i, j))

# Set decision variable y for each pair of nodes i and k, and each district j
for i in range(num_counties):
    for k in range(num_counties):
        for j in range(num_districts):
            y = m.addVar(vtype=gp.GRB.BINARY, name="y_{}_{}_{}".format(i, k, j))

# Set slack variable
for j in range(num_districts):
    slack = m.addVar(obj = 1, name = "slack")



In [53]:
d = pd.read_csv("districting_2020\\2020\\{}\\counties\\graph\\{}_distances.csv".format(state, state))
d = d.iloc[0:, 1:]
d[[0,1]]

KeyError: "None of [Index([0, 1], dtype='int32')] are in the [columns]"

In [28]:
m.setObjective(gp.quicksum(y[i,k,j] * d[i,k] for i in range(num_counties) for k in range(num_counties) for j in range(num_districts)) + gp.quicksum(slack[j] for j in range(num_districts)), GRB.MINIMIZE)
    , GRB.MAXIMIZE)

m.addConstrs((gp.quicksum(nutritionValues[f, c] * buy[f] for f in foods)
             == [minNutrition[c], maxNutrition[c]]
             for c in categories), "_")


TypeError: 'Var' object is not iterable